In [133]:
#imports
import csv
import pandas as pd
import numpy as np
import math

In [134]:
# df = pd.read_csv('12.csv')
df = pd.read_csv("12_final_processed.csv")

In [135]:
print("================== EWMA ===================")

# Separating out data for EWMA:

df_august = df[(df['Date'] >= '2020-08-01') & (df['Date'] <= '2020-08-28')]

ds_MI_cases, ds_MN_cases, ds_MI_deaths, ds_MN_deaths = df_august['MI confirmed'], df_august['MN confirmed'], df_august['MI deaths'], df_august['MN deaths']

================== EWMA ===================


In [136]:
'''
Calculates ewma for the first n values of given dataset and alpha
'''
def ewma(alpha, ds, n):
    estimates = [] # predictions
    # y hat 1 = starting point
    estimates.append(ds.iloc[0])
    for i in range(1,n):
        newVal = alpha*ds.iloc[i-1] + (1 - alpha)*estimates[i-1]
        estimates.append(int(newVal))
    # print(estimates)
    return pd.Series(estimates)

'''
Predicts values for the last week for given dataset of 4 weeks and alpha = 0.5
'''
def ewma_5(ds):
    ewma_over_august = ewma(0.5, ds, 28)
    return ewma_over_august[-7:]

'''
Predicts values for the last week for given dataset of 4 weeks and alpha = 0.8
'''
def ewma_8(ds):
    ewma_over_august = ewma(0.8, ds, 28)
    return ewma_over_august[-7:]


In [137]:

ds_MI_cases_predicted_5 = ewma_5(ds_MI_cases)
ds_MI_cases_predicted_8 = ewma_8(ds_MI_cases)

print("\nMI cases predictions: ")
print("\nAlpha = 0.5 :")
print("Days  Values")
print(ds_MI_cases_predicted_5)
print("\nAlpha = 0.8 :")
print("Days  Values")
print(ds_MI_cases_predicted_8)

ds_MN_cases_predicted_5 = ewma_5(ds_MN_cases)
ds_MN_cases_predicted_8 = ewma_8(ds_MN_cases)

print("\nMN cases predictions: ")
print("\nAlpha = 0.5 :")
print("Days  Values")
print(ds_MN_cases_predicted_5)
print("\nAlpha = 0.8 :")
print("Days  Values")
print(ds_MN_cases_predicted_8)

ds_MI_deaths_predicted_5 = ewma_5(ds_MI_deaths)
ds_MI_deaths_predicted_8 = ewma_8(ds_MI_deaths)

print("\nMI deaths predictions: ")
print("\nAlpha = 0.5 :")
print("Days  Values")
print(ds_MI_deaths_predicted_5)
print("\nAlpha = 0.8 :")
print("Days  Values")
print(ds_MI_deaths_predicted_8)

ds_MN_deaths_predicted_5 = ewma_5(ds_MN_deaths)
ds_MN_deaths_predicted_8 = ewma_8(ds_MN_deaths)

print("\nMN deaths predictions: ")
print("\nAlpha = 0.5 :")
print("Days  Values")
print(ds_MN_deaths_predicted_5)
print("\nAlpha = 0.8 :")
print("Days  Values")
print(ds_MN_deaths_predicted_8)





MI cases predictions: 

Alpha = 0.5 :
Days  Values
21    296
22    861
23    812
24    845
25    898
26    870
27    866
dtype: int64

Alpha = 0.8 :
Days  Values
21     111
22    1163
23     843
24     871
25     935
26     861
27     862
dtype: int64

MN cases predictions: 

Alpha = 0.5 :
Days  Values
21    715
22    724
23    720
24    717
25    563
26    546
27    850
dtype: int64

Alpha = 0.8 :
Days  Values
21     791
22     745
23     722
24     715
25     470
26     517
27    1026
dtype: int64

MI deaths predictions: 

Alpha = 0.5 :
Days  Values
21     6
22    13
23     8
24     6
25    13
26     9
27    12
dtype: int64

Alpha = 0.8 :
Days  Values
21     2
22    17
23     6
24     4
25    17
26     8
27    14
dtype: int64

MN deaths predictions: 

Alpha = 0.5 :
Days  Values
21     8
22     8
23     7
24     5
25     6
26    10
27    11
dtype: int64

Alpha = 0.8 :
Days  Values
21     8
22     8
23     6
24     4
25     7
26    12
27    12
dtype: int64


In [138]:
print("================== AR ===================")

================== AR ===================


In [139]:
'''
Generates X and Y arrays for given dataset for AR
'''
def generateXnY(data, p):
    x = []
    y = []
    t = len(data)
    for i in range(p, t):
        l = [1]
        l.extend(np.flip(data[i-p:i]))
        x.append(l)
        y.append(data[i])
    return np.array(x), np.array(y)


'''
generates list of beta coefficients for given X and Y
'''
def calc_beta_coeffs(x, y):
    x_trans = x.transpose()
    part1 = np.matmul(x_trans, x)
    part1_inverse = np.linalg.inv(part1)
    part2 = np.matmul(part1_inverse, x_trans)
    return np.matmul(part2, y)


'''
performs autoregression with p as order of AR, 
'''
def ar(p, ds_3_weeks):
    #     three_weeks_data = np.array(ds[:21])
    #     b = np.array(ds[21:])
    
    #     ds_3_weeks = np.array(ds_3_weeks)
    initX, initY = generateXnY(ds_3_weeks, p)
    
    betas = calc_beta_coeffs(initX, initY)
    
    y_predicted = []
    for i in range(7):
        
        y_prev = ds_3_weeks[-p:]
        y_prev.append(1)
        
        y_hat = 0
        for j in range(p+1):
            y_hat = y_hat + betas[j]*y_prev[p-j]
        # y_hat = betas[0] + betas[1]*y_prev[2] + betas[2]*y_prev[1] + betas[3]*y_prev[0]
        
        ds_3_weeks.append(y_hat)
        
        y_predicted.append(y_hat)
        intermediateX, intermediateY = generateXnY(ds_3_weeks, p)
        betas = calc_beta_coeffs(intermediateX, intermediateY)
        
    return [int(x) for x in y_predicted]
    


In [140]:
ar_training_df = df[(df['Date'] >= '2020-08-01') & (df['Date'] <= '2020-08-21')]
ar_predict_df = df[(df['Date'] >= '2020-08-22') & (df['Date'] <= '2020-08-28')]

In [141]:
ds_MI_cases_AR_3 = ar(3, ar_training_df['MI confirmed'].tolist())
ds_MI_cases_AR_5 = ar(5, ar_training_df['MI confirmed'].tolist())

ds_MN_cases_AR_3 = ar(3, ar_training_df['MN confirmed'].tolist())
ds_MN_cases_AR_5 = ar(5, ar_training_df['MN confirmed'].tolist())

ds_MI_deaths_AR_3 = ar(3, ar_training_df['MI deaths'].tolist())
ds_MI_deaths_AR_5 = ar(5, ar_training_df['MI deaths'].tolist())

ds_MN_deaths_AR_3 = ar(3, ar_training_df['MN deaths'].tolist())
ds_MN_deaths_AR_5 = ar(5, ar_training_df['MN deaths'].tolist())


print("\nMI cases predictions: ")
print("\np = 3 :")
print(ds_MI_cases_AR_3)
print("\np = 5 :")
print(ds_MI_cases_AR_5)

print("\nMN cases predictions: ")
print("\np = 3 :")
print(ds_MN_cases_AR_3)
print("\np = 5 :")
print(ds_MN_cases_AR_5)

print("\nMI deaths predictions: ")
print("\np = 3 :")
print(ds_MI_deaths_AR_3)
print("\np = 5 :")
print(ds_MI_deaths_AR_5)

print("\nMN deaths predictions: ")
print("\np = 3 :")
print(ds_MN_deaths_AR_3)
print("\np = 5 :")
print(ds_MN_deaths_AR_5)


MI cases predictions: 

p = 3 :
[545, 603, 805, 719, 719, 660, 671]

p = 5 :
[400, 450, 674, 528, 548, 508, 587]

MN cases predictions: 

p = 3 :
[713, 614, 550, 600, 662, 696, 672]

p = 5 :
[749, 704, 572, 547, 556, 655, 702]

MI deaths predictions: 

p = 3 :
[10, 10, 12, 10, 10, 10, 10]

p = 5 :
[17, 8, 14, 10, 13, 8, 12]

MN deaths predictions: 

p = 3 :
[12, 8, 7, 9, 8, 7, 8]

p = 5 :
[11, 8, 7, 9, 9, 7, 8]


In [142]:

def calc_mse(y_hats, ys, cnt):
    return np.sum((ys[-cnt:] - y_hats)**2)/cnt

In [143]:
def calc_mape(ys, y_hats, cnt):
    mape_error = 0 #initialized to zero
    for y_i, y_hat_j in zip(ys[-cnt:], y_hats):
        if y_i != 0: #to handle zero division error when actual data point is zero
            mape_error = mape_error + (abs(y_i - y_hat_j) / y_i * 100)
    mape_error = mape_error / cnt
    return mape_error


In [144]:
print("================= Errors - MSE =================")


MSE_MI_cases_5 = calc_mse(ds_MI_cases_predicted_5, ar_predict_df['MI confirmed'], 7)
MSE_MI_cases_8 = calc_mse(ds_MI_cases_predicted_8, ar_predict_df['MI confirmed'], 7)

print("\nMSE - MI cases: ")
print("\nAlpha = 0.5 :")
print(MSE_MI_cases_5)
print("\nAlpha = 0.8 :")
print(MSE_MI_cases_8)


MSE_MN_cases_5 = calc_mse(ds_MN_cases_predicted_5, ar_predict_df['MN confirmed'], 7)
MSE_MN_cases_8 = calc_mse(ds_MN_cases_predicted_8, ar_predict_df['MN confirmed'], 7)

print("\nMSE - MN cases: ")
print("\nAlpha = 0.5 :")
print(MSE_MN_cases_5)
print("\nAlpha = 0.8 :")
print(MSE_MN_cases_8)


MSE_MI_deaths_5 = calc_mse(ds_MI_deaths_predicted_5, ar_predict_df['MI deaths'], 7)
MSE_MI_deaths_8 = calc_mse(ds_MI_deaths_predicted_8, ar_predict_df['MI deaths'], 7)

print("\nMSE - MI deaths: ")
print("\nAlpha = 0.5 :")
print(MSE_MI_deaths_5)
print("\nAlpha = 0.8 :")
print(MSE_MI_deaths_8)


MSE_MN_deaths_5 = calc_mse(ds_MN_deaths_predicted_5, ar_predict_df['MN deaths'], 7)
MSE_MN_deaths_8 = calc_mse(ds_MN_deaths_predicted_8, ar_predict_df['MN deaths'], 7)

print("\nMSE - MN deaths: ")
print("\nAlpha = 0.5 :")
print(MSE_MN_deaths_5)
print("\nAlpha = 0.8 :")
print(MSE_MN_deaths_8)



================= Errors - MSE =================

MSE - MI cases: 

Alpha = 0.5 :
0.0

Alpha = 0.8 :
0.0

MSE - MN cases: 

Alpha = 0.5 :
0.0

Alpha = 0.8 :
0.0

MSE - MI deaths: 

Alpha = 0.5 :
0.0

Alpha = 0.8 :
0.0

MSE - MN deaths: 

Alpha = 0.5 :
0.0

Alpha = 0.8 :
0.0


In [145]:
print("================= Errors - MAPE =================")


MAPE_MI_cases_5 = calc_mape(ds_MI_cases_predicted_5, ds_MI_cases, 7)
MAPE_MI_cases_8 = calc_mape(ds_MI_cases_predicted_8, ds_MI_cases, 7)

print("\nMAPE - MI cases: ")
print("\nAlpha = 0.5 :")
print(MAPE_MI_cases_5)
print("\nAlpha = 0.8 :")
print(MAPE_MI_cases_8)


MAPE_MN_cases_5 = calc_mape(ds_MN_cases_predicted_5, ds_MN_cases, 7)
MAPE_MN_cases_8 = calc_mape(ds_MN_cases_predicted_8, ds_MN_cases, 7)

print("\nMAPE - MN cases: ")
print("\nAlpha = 0.5 :")
print(MAPE_MN_cases_5)
print("\nAlpha = 0.8 :")
print(MSE_MN_cases_8)


MAPE_MI_deaths_5 = calc_mape(ds_MI_deaths_predicted_5, ds_MI_deaths, 7)
MAPE_MI_deaths_8 = calc_mape(ds_MI_deaths_predicted_8, ds_MI_deaths, 7)

print("\nMAPE - MI deaths: ")
print("\nAlpha = 0.5 :")
print(MAPE_MI_deaths_5)
print("\nAlpha = 0.8 :")
print(MAPE_MI_deaths_8)


MAPE_MN_deaths_5 = calc_mape(ds_MN_deaths_predicted_5, ds_MI_deaths, 7)
MAPE_MN_deaths_8 = calc_mape(ds_MN_deaths_predicted_8, ds_MI_deaths, 7)

print("\nMAPE - MN deaths: ")
print("\nAlpha = 0.5 :")
print(MAPE_MN_deaths_5)
print("\nAlpha = 0.8 :")
print(MAPE_MN_deaths_8)



================= Errors - MAPE =================

MAPE - MI cases: 

Alpha = 0.5 :
39.18825711506203

Alpha = 0.8 :
103.06784124336295

MAPE - MN cases: 

Alpha = 0.5 :
19.365797899709953

Alpha = 0.8 :
0.0

MAPE - MI deaths: 

Alpha = 0.5 :
68.8949938949939

Alpha = 0.8 :
107.72308923569427

MAPE - MN deaths: 

Alpha = 0.5 :
62.23716759431045

Alpha = 0.8 :
51.785714285714285
